In [41]:
import os
import cv2
import pickle
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from keras import layers

In [42]:
np.random.seed(1000)
# ignore information messgaes from tensorflow, but we will receieve error messages
os.environ['TFF_CPP_MIN_LOG_LEVEL'] = '2'

%matplotlib inline

In [43]:
pickle_in = open("./resources/X_train.pickle","rb")
X_train = pickle.load(pickle_in)

pickle_in = open("./resources/Y_train.pickle","rb")
Y_train = pickle.load(pickle_in)

pickle_in = open("./resources/X_valid.pickle","rb")
X_valid = pickle.load(pickle_in)

pickle_in = open("./resources/Y_valid.pickle","rb")
Y_valid = pickle.load(pickle_in)

pickle_in = open("./resources/X_test.pickle","rb")
X_test = pickle.load(pickle_in)

pickle_in = open("./resources/Y_test.pickle","rb")
Y_test = pickle.load(pickle_in)

print(f"X_train= {X_train.shape} Y_train= {Y_train.shape}")
print(f"X_valid= {X_valid.shape} Y_valid= {Y_valid.shape}")
print(f"X_test= {X_test.shape} Y_test= {Y_test.shape}")

X_train= (312, 512, 512, 3) Y_train= (312,)
X_valid= (180, 512, 512, 3) Y_valid= (180,)
X_test= (138, 512, 512, 3) Y_test= (138,)


In [44]:
mean_img = np.mean(X_train, axis=0)

In [45]:
DATA_DIR = 'C:\\Users\\Admin\\Desktop\\New folder (2)\\data_dir'
TRAIN_DATA_DIR = os.path.join(DATA_DIR, 'train')

In [46]:
CATEGORIES = []
for i in os.listdir(TRAIN_DATA_DIR):
    CATEGORIES.append(i)

In [47]:
print(CATEGORIES)

['ABDULLAH AL SHAHRIAR', 'AHMED IMTIAZ', 'AL-NAFI', 'ASHIK AHAMED', 'BISHANATH TARAFDER', 'ESM-E MOULA CHOWDHURY ABHA', 'FAHIM RAHMAN', 'FAIZA BINTE ZAMAN', 'IMAM HASAN JAMI', 'MD ABU ZAYED KHAN', 'MD ATIK ULLAH KHAN', 'MD FARDIN AMIN RIYAD', 'MD IMTIAJ ALAM SAJIN', 'MD JAHID HASSAN', 'MD MUNTASIR AREFIN NAEEM', 'MD NAIMUR RAHMAN', 'MD SAJID ISLAM KHAN', 'MD SHANZID HASAN', 'MD TAREK MAHMUD', 'MD TOYABUR RAHAMAN', 'MD WAHIDUZZAMAN SUVA', 'MOHAMMAD BIN AB JALIL SHEAKH', 'MOHAMMAD NUR', 'MOHAMMED TANVIR HASSAN', 'NAFIS MUBASSHIR SHAH', 'NAHAR ISLAM NISHI', 'NAVID MAHFUZ NAYEEM', 'RAHAD-UL-ISLAM RABBY', 'REZWAN AHMAD', 'S M FAISAL', 'SABBIR AHMED', 'SADAT BIN MASUD', 'SATYAJIT DAS', 'SHEIKH AKIB ALMAS', 'SRABONE RAXIT', 'SYEDA HUMAIRA JABEEN', 'TAHFIM IBN KHAN', 'TAHMID AL RAFID SIDDIQUE', 'TARIKUL ISLAM NISHAT']


In [48]:
# Load the pre-trained model
model = tf.keras.models.load_model("./resources/alexnet_model.h5", compile=False)

# Function to preprocess the face image
def preprocess(face_img):
    face_img = cv2.resize(face_img, (512, 512))  # Resize to the size expected by your model
    face_img = face_img-mean_img
    face_img = np.expand_dims(face_img, axis=0)
    return face_img

# Start video capture from webcam
cap = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        break

    face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    faces = face_cascade.detectMultiScale(frame, 1.1, 4)

    # Loop through detected faces
    for (x, y, w, h) in faces:
        face_img = frame[y:y+h, x:x+w]
        processed_face = preprocess(face_img)

        # Predict mask status
        prediction = model.predict(processed_face)
        label = int(np.argmax(prediction, axis=1))
        
        # Label the prediction
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, CATEGORIES[label], (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
                

    # Display the resulting frame
    cv2.imshow('Face Detection', frame)

    # Break the loop with the 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 101ms/step
